# Dependencies

In [9]:
from pathlib import Path

import cv2
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

In [10]:
output_path = Path('../temp/videos')
output_path.mkdir(parents=True, exist_ok=True)

# Utilities

In [11]:
def group_neighbour_flows(flow: np.ndarray, window_size: int = 4) -> np.ndarray:

    # a single frame
    if flow.ndim == 3:
        new_shape = (1, flow.shape[0] // window_size, flow.shape[1] // window_size, 2)

    # multiple frames
    elif flow.ndim == 4:
        new_shape = (flow.shape[0], flow.shape[1] // window_size, flow.shape[2] // window_size, 2)

    new_flow = np.zeros(new_shape, dtype=flow.dtype)

    # down sample flow
    for i in range(0, flow.shape[1], window_size):
        for j in range(0, flow.shape[2], window_size):
            new_flow[:, i // window_size, j // window_size] = np.mean(flow[:, i:i+window_size, j:j+window_size], axis=(1, 2))

    return new_flow

# Load a video

In [12]:
cap = cv2.VideoCapture("https://media.xiph.org/video/derf/y4m/tt_sif.y4m")

fps = round(cap.get(cv2.CAP_PROP_FPS))
frames = []

while (cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frames.append(frame)
    else:
        break

cap.release()

In [13]:
# color space: BGR
frames = np.array(frames)

# log
print(f"video.shape: {frames.shape}")
print(f"video.dtype: {frames.dtype}")
print(f"type(video): {type(frames)}")
print(f"frame per second (fps): {fps}")

video.shape: (112, 240, 352, 3)
video.dtype: uint8
type(video): <class 'numpy.ndarray'>
frame per second (fps): 30


In [14]:
gray_frames = np.array([cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames])

# [Optical FLow](https://www.mathworks.com/discovery/optical-flow.html)
- Definition
   - It refers to the apparent motion of objects, surfaces, and edges in a visual scene caused by the relative motion between an observer (camera) and the scene.
- Classic methods:
   - [Lucas-Kanade](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323)
   - [Farneback](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af)

## Dense Optical Flow using Farneback method

In [15]:
flows = np.zeros(shape=(frames.shape[0] - 1, frames.shape[1], frames.shape[2], 2), dtype=np.float32)

for i in range(len(frames) - 1):
    flows[i] = cv2.calcOpticalFlowFarneback(
        prev=gray_frames[i],
        next=gray_frames[i + 1],
        flow=None,
        pyr_scale=.5,
        levels=3,
        winsize=7,
        iterations=3,
        poly_n=7,
        poly_sigma=1.5,
        flags=0
    )


window_size = 8
grouped_flows = group_neighbour_flows(flows, window_size)

In [ ]:
# plot
start_frame = 0
fig = plt.figure(figsize=(16, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 2)

ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(cv2.cvtColor(frames[start_frame], cv2.COLOR_BGR2RGB))
ax1.set_title(f"frame: {start_frame}")
ax1.axis('off')

ax2 = fig.add_subplot(gs[1, 0])
ax2.imshow(cv2.cvtColor(frames[start_frame], cv2.COLOR_BGR2RGB))
ax2.set_title(f"frame: {start_frame + 1}")
ax2.axis('off')

x, y = np.meshgrid(range(grouped_flows.shape[2]), range(grouped_flows.shape[1]))
ax3 = fig.add_subplot(gs[:, 1])
ax3.quiver(x, y, grouped_flows[start_frame, :, :, 0], grouped_flows[start_frame, :, :, 1], angles='xy', scale_units='xy', scale=1, color='k')
ax3.set_title(f"motion: {start_frame} -> {start_frame+1}")
ax3.invert_yaxis()

plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(16, 8), layout='tight')
gs = gridspec.GridSpec(nrows=2, ncols=2, width_ratios=[1, 2])

ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[1, 0])
ax2 = plt.subplot(gs[:, 1])

# create a grid of x, y coordinates
height, width = grouped_flows.shape[1:3]
x, y = np.meshgrid(range(width), range(height))


# updates the figure frame by frame
def update(i):
    for ax in fig.axes:
        ax.cla()
        ax.axis('off')

    ax0.imshow(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB))
    ax0.set(title=f"frame: {i + 1}")

    ax1.imshow(cv2.cvtColor(frames[i + 1], cv2.COLOR_BGR2RGB))
    ax1.set(title=f"frame: {i + 2}")

    ax2.quiver(x, y, grouped_flows[i, :, :, 0], grouped_flows[i, :, :, 1], angles='xy', scale_units='xy', scale=1, color='k')
    ax2.invert_yaxis()
    ax2.set(title=f"frame {i + 1}")


# create an animation
ani = animation.FuncAnimation(fig, update, frames=len(frames) - 1)

# save the animation as a video file
ani.save(filename=f"{output_path}/optical_flow_1.mp4", writer='ffmpeg', fps=fps // 4)

## Sparse Optical Flow using Lucas-Kanade method